In [2]:
# The code was removed by DSX for sharing.

<a id="principio"></a> 
<img src="https://www.dsi.uclm.es/personal/LuisDeLaOssa/muii/logoD.png" alt="Logo MUII" align="right">

<br><br>


## Práctica 7

# Algoritmo de filtrado colaborativo en Spark


<br>
<div style="text-align: right">
Luis de la Ossa
<br>
Master Universitario en Ingeniería Informática
<br>
Universidad de Castilla-La Mancha

</div>

---
<table cellspacing="0" cellpadding="0" border="0">
    <tbody>
        <tr>
            <td rowspan="2">
                <div><img height="317" width="214" alt="io" title="io" style="background-repeat: no-repeat;backgroundposition: 50%;border-radius: 50%;background-size: 100% auto;" src="https://media.licdn.com/dms/image/C4E03AQED9j4Ps-ibhQ/profile-displayphoto-shrink_200_200/0?e=1533168000&v=beta&t=KeYKeXKTbF7StW-KHF8-o_0zXm-P6RM1vwuLdGFD8tU" </div>
            </td>
            <td>
                <h1 style="color:#113D68"><span>Héctor Fernando Bahamonde Rivera</span></h1>
                <hr>
                <center><i class="fa fa-envelope" style="color:#113D68"></i></center> 
                <h2><a href="mailto:HectorFernando.Bahamonde@alu.uclm.es" target="_top">HectorFernando.Bahamonde@alu.uclm.es</a></h2>
            </td>
        </tr>
   </tbody>
</table>

In [3]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
# Algunas inconsistencias con las versiones dan lugar a avisos molestos. Se ignoran.
import warnings
warnings.filterwarnings('ignore')

Antes de empezar, se ha de comprobar que está activo el contexto Spark.

In [4]:
sc

---

## 1. Introducción 

<br>
El objetivo de esta práctica es familiarizarse con la programación y tratamiento de datos en _Apache Spark_. Para ello, se implementará un algoritmo de recomendación basado en filtrado colaborativo. En concreto, basado en ítems más cercanos.

Hay que tener en cuenta que _Spark_ hace las operaciones cuando necesita los datos, y no antes. Por eso, cuando se reportan los errores, éstos pueden haberse producido en celdas anteriores aparentemente correctas. Con el fin de que podáis comprobar cada paso, se ha añadido una llamada a la función `take()` comentada. 

<div class="alert alert-block alert-info">
<i class="fa fa-info-circle" aria-hidden="true"></i> El fin de este trabajo es esencialmente didáctico. Por tanto, no todos los pasos están optimizados. Además, Spark induce una sobrecarga que, para este problema concreto (con estos datos), no compensa, y produce una degradación del rendimiento con respecto a la implementación vectorizada con Numpy/Sklearn.
</div>

---

## 2. Lectura de datos

Para esta práctica, al igual que en la anterior, usaremos un conjunto de datos de [Movielens](https://grouplens.org/datasets/movielens/latest/). En concreto, se trabajará con el conjunto pequeño, almacenado en los archivos  `movies.csv` y `ratings.csv`.

Para leerlos, es necesario proporcionar una ruta de acceso, que dependerá de si se trabaja en modo local, o en el entorno DSX de IBM. 

<div class="alert alert-block alert-warning">
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Las siguientes celdas almacenan la ruta de acceso a cada uno de los archivos para el trabajo en modo local. En caso de trabajar en DSX, han de ser generadas automáticamente desde el propio entorno.
</div>

A continuación, y puesto que la lista de películas solamente se utilizará en modo local, la almacenaremos en un _DataFrame_ Pandas.

In [5]:
import pandas as pd
import numpy as np

# Acceso al conjunto de datos con la información de las películas. 
#path_movies = "./datos/movies.csv"

# Acceso al conjunto de datos +con la información de las películas desde DSX
path_movies = project.get_file('movies_big.csv')

In [6]:
df_movies = pd.read_csv(path_movies, sep=',', index_col=0)
# Muestra las dos primeras y las dos últimas
display(df_movies.iloc[[0,1, -2,-1]])
# Número de películas (no siguen una secuencia)
print("Número de películas: ",len(df_movies))

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
176275,Satan Triumphant (1917),(no genres listed)
176279,Queerama (2017),(no genres listed)


Número de películas:  45843


Los datos relativos a las votaciones se almacenarán en un RDD.

In [7]:
path_file = project.get_file('ratings.csv')
ratings = []
for line in path_file:
    ratings.append(str(line).replace("b'","").replace("\\r\\n'",""))

In [8]:
dataRDD = sc.parallelize(ratings) #textFile('ratings.csv')
dataRDD.take(3)
# ['1,31,2.5,1260759144', '1,1029,3.0,1260759179', '1,1061,3.0,1260759182']

['1,31,2.5,1260759144', '1,1029,3.0,1260759179', '1,1061,3.0,1260759182']

---

## 3. Algoritmo de recomendación basado en similaridad entre películas.

En esta parte implementaremos un algoritmo basado en similaridad entre ítems. Posteriormente, se obtendrán las películas similares a una dada. 

La técnica es parecida a la vista en clase. Dadas dos películas, su similaridad se obtiene a partir de las valoraciones que han hecho los usuarios que hayan visto ambas. Hemos de situarnos en un contexto en el que el número de usuarios es muy grande. 

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 1 </font> 
En primer lugar, vamos a convertir el RDD de texto en un RDD con tuplas `(user_id, (movie_id, rating))`, denominado `ratingsRDD`, en el que `user_id` y `movie_id` son enteros, y `rating`es float. Utilizar la función `map`.

In [9]:
ratingsRDD = dataRDD.map(lambda x: (int(x.split(',')[0]),(int(x.split(',')[1]),float(x.split(',')[2]))))

ratingsRDD.take(3)
#[(1, (31, 2.5)), (1, (1029, 3.0)), (1, (1061, 3.0))]

[(1, (31, 2.5)), (1, (1029, 3.0)), (1, (1061, 3.0))]

De cara a diseñar el algoritmo, el principal factor a tener en cuenta es que se van a comparar pares de películas, y para ello solamente se considerán las valoraciones de usuarios que hayan visto ambas. Por ejemplo, si dos películas `movie_A`y `movie_B` han sido valoradas por cinco usuarios,

$$
movie\_A = [ 0, 2, 4, 0, 1] \quad movie\_B = [1 ,3, 5, 0, 0], 
$$

solamente se se utilizarán las valoraciones de los usuarios 2 y 3, que han visto ambas películas. Visto de otro modo, más aproximado a cómo se procederá aquí, el usuario 1, por ejemplo, no se tendrá en cuenta para calcular la distancia entre la película `movie_A` y ninguna otra. Por ello, en primer lugar se van a generar los pares de películas vistas por un mismo usuario. 

En el apartado anterior se han generado tuplas del tipo `(user_id, (movie_id, rating))`. Si se hace un _inner join_ de `ratingsRDD` consigo mismo, cada entrada del primer RDD se unirá con las entradas del segundo RDD con las que comparta clave (`user_id`), generando tuplas del tipo:  `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`.



#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 2 </font> 

Hacer un join del RDD `ratingsRDD` y almacenar el resultado en otro denominado `join_ratingsRDD`.

In [10]:
join_ratingsRDD = ratingsRDD.join(ratingsRDD)


join_ratingsRDD.take(3)
#[(256, ((3, 3.0), (3, 3.0))),
# (256, ((3, 3.0), (5, 3.0))),
# (256, ((3, 3.0), (6, 3.0)))]

[(256, ((3, 3.0), (3, 3.0))),
 (256, ((3, 3.0), (5, 3.0))),
 (256, ((3, 3.0), (6, 3.0)))]

En este punto, cada par de películas vistas por un usuario aparece dos veces en `join_ratingsRDD`, ya que también aparecerían en orden inverso. Es decir, por cada entrada `(user_id, ((movie_id1, rating1), (movie_id2, rating2))` tendríamos otra como `(user_id, ((movie_id2, rating2), (movie_id1, rating1))`. Para eliminar los duplicados, vamos a dejar solamente aquellas entradas en las que el índice de la primera película `movie_id1`, sea menor que el de la segunda `movie_id2`.


#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 3 </font> 

Implementar una función denominada `filter_duplicates` que reciba una tupla del tipo `((movie_id1, rating1), (movie_id2, rating2))` y devuelva `True` cuando `movie_id1 < movie_id2`, y `False` en caso contrario. 

In [11]:
def filter_duplicates(ratings): 
    return (ratings[1][0][0] < ratings[1][1][0])
filter_duplicates((256, ((223, 3.0), (43, 3.0))))

False

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 4 </font> 

Utilizar la función anterior para eliminar los duplicados de `join_ratingsRDD`, mediante `filter`. 

<div class="alert alert-block alert-warning">
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Cada entrada de `join_ratingsRDD` es una tupla `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`. Por tanto, a la función `filter_duplicates` hay que pasarle el segundo de los componentes de la tupla, ya que el primero es `user_id`.
</div>

In [12]:
join_ratingsRDD = join_ratingsRDD.filter(lambda x: filter_duplicates(x))
 

join_ratingsRDD.take(3)
#[(256, ((3, 3.0), (5, 3.0))),
# (256, ((3, 3.0), (6, 3.0))),
# (256, ((3, 3.0), (7, 3.0)))]

[(256, ((3, 3.0), (5, 3.0))),
 (256, ((3, 3.0), (6, 3.0))),
 (256, ((3, 3.0), (7, 3.0)))]


En este punto, se dispone de entradas `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`. De cara a computar la similaridad entre películas, se incluirán en los vectores correspondientes a las películas `movie_id1` y `movie_id2` las valoraciones correspondientes al usuario `user_id`.  Sin embargo, el usuario en sí es irrelevante. Por otra parte, de cara a hacer los cálculos de similaridad, se han de considerar pares de películas valoradas por un mismo usuario. 

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 5 </font> 

Transformar el RDD `join_ratingsRDD` en el que las entradas son del tipo,  `(user_id, ((movie_id1, rating1), (movie_id2, rating2))`, en un RDD pareado denominado `movie_pairsRDD`, en el que las entradas sean de tipo `((movie_id1, movie_id2),(rating1, rating2))`. Utilizar la función `map`.

In [13]:
movie_pairsRDD = join_ratingsRDD.map(lambda x: (x[1][0],x[1][1]))


movie_pairsRDD.take(3)
#[((3, 5), (3.0, 3.0)), ((3, 6), (3.0, 3.0)), ((3, 7), (3.0, 3.0))]

[((3, 3.0), (5, 3.0)), ((3, 3.0), (6, 3.0)), ((3, 3.0), (7, 3.0))]

<div class="alert alert-block alert-danger">
<i class="fa fa-exclamation-triangle" aria-hidden="true"></i>
__OJO!__ Esta última operación genera un RDD de gran tamaño. Algunas operaciones sobre éste pueden requerir varios minutos. 
</div>

In [13]:
movie_pairsRDD.count()
#25313236

25313236

Para cada poder calcular la similaridad entre un par de películas, han de obtenerse las valoraciones echas por cada usuario que haya visto ambas, es decir, buscar todas las tuplas `(movie_id1, movie_id2)`, y agrupar los pares `(rating1, rating2)` de cada una de ellas.

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 6 </font> 

Agrupar las valoraciones existentes en el RDD `movie_pairsRDD` para cada par `(movie_id1, movie_id2)`. Se debe generar un RDD pareado denominado `movie_pairs_ratingsRDD` del tipo `(movie_id1, movie_id2): [(rating1, rating2), (rating1, rating2), ...] `.  Utilizar para ello `groupByKey()`.

In [14]:
movie_pairs_ratingsRDD = movie_pairsRDD.map(lambda x: ((x[0][0],x[1][0]),(x[0][1],x[1][1]))).groupByKey()


# Muestra las tres primeras entradas.
movie_pairs_ratingsRDD.take(3)

# Este resultado puede cambiar. 
#[((5152, 156726), <pyspark.resultiterable.ResultIterable at 0x7fbdd8d2b3c8>),
# ((3476, 5846), <pyspark.resultiterable.ResultIterable at 0x7fbdd8d2b438>),
# ((3704, 90430), <pyspark.resultiterable.ResultIterable at 0x7fbdd8d2bf28>)]

[((160, 950), <pyspark.resultiterable.ResultIterable at 0x7f7b66be9e10>),
 ((3727, 67255), <pyspark.resultiterable.ResultIterable at 0x7f7b66be9eb8>),
 ((5152, 156726), <pyspark.resultiterable.ResultIterable at 0x7f7b66be9f28>)]


---

### Cálculo de la similaridad coseno. 

Para cada par de películas, el RDD `movie_pairs_ratingsRDD` contiene una lista de tuplas, con formato,`[(3.0, 4.0), (2.0, 1.0), (3.0, 1.0)]`,  en la que cada tupla representa la valoración que ha hecho un mismo usuario a ambas. Es decir, la lista del ejemplo correspondería al par de vectores `[(3.0, 2.0, 1.0),(4.0, 1.0, 1.0)]`. A partir de esta información, es posible calcular la similaridad coseno. Además, se devolverá el número de usuarios que ha puntuado las dos películas.

In [15]:
from math import sqrt

def cos_sim(ratings):
    num_ratings = 0
    sum_1 = sum_2= sum_12 = 0    
    for rating1, rating2 in ratings:
        sum_1 += rating1 * rating1
        sum_2 += rating2 * rating2
        sum_12 += rating1 * rating2
        num_ratings += 1
    
    num = sum_12
    den = sqrt(sum_1) * sqrt(sum_2)

    return num / float(den), num_ratings

cos_sim([(3.0, 4.0), (2.0, 1.0), (3.0, 1.0)])
#(0.8542821429703302, 3)

(0.8542821429703302, 3)

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 7 </font> 

Calcular la similaridad coseno para cada par de películas de almacenadas en `movie_pairs_ratingsRDD`, y almacenar el resultado en `movie_pair_similaritiesRDD`. Hacer persistente el resultado llamando al método `cache()`. El resultado incluye la similaridad, así como el número de personas que han valorado ambas películas (es lo que devuelve la función `cos_sim`).

In [16]:
movie_pair_similaritiesRDD  = movie_pairs_ratingsRDD.map(lambda x: (x[0],cos_sim(list(x[1])))).cache()


movie_pair_similaritiesRDD.take(3)

# Puede cambiar
#[((1757, 52241), (1.0, 1)),
# ((1639, 67087), (0.8497058314499201, 5)),
# ((2991, 101283), (1.0, 1))]

[((160, 950), (0.9171382948141402, 4)),
 ((3727, 67255), (1.0, 1)),
 ((5152, 156726), (1.0, 1))]

---

## 4. Películas similares a una dada.

A continuación, se van a generar las películas similares a una dada (la 10). Solamente se considerarán aquellas cuya similaridad esté por encima de 0.97 y, además, hayan sido votadas por al menos 20 personas que votaron la película original. 

In [17]:
min_sim = 0.97
min_common = 20
movie_id = 1


#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 8 </font> 

Filtrar los resultados de interés. Es decir, lo que que cumplan los criterios descritos anteriormente y almacenarlos en un RDD denominado `similar_moviesRDD`, es decir, que la primera o la segunda película sea `movie_id`, que la similaridad esté por encima de 0.97, y que el número de personas que han visto ambas sea al menos 20.  Utilizar la función `filter`.

In [18]:
similar_moviesRDD = movie_pair_similaritiesRDD.filter(lambda x: (movie_id in x[0]) and (min_sim < x[1][0]) and (min_common <= x[1][1]))


similar_moviesRDD.take(3)
#[((10, 168), (0.9820054554740041, 24)),
# ((10, 368), (0.9704045420289952, 36)),
# ((10, 180), (0.9790187094625074, 21))]

[((1, 76093), (0.9714706917311978, 23)),
 ((1, 29), (0.975119822819981, 26)),
 ((1, 34405), (0.9721097486997603, 27))]

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 9 </font> 

Ordenar los resultados de `similar_moviesRDD` por similaridad. Utilizar para ello `sortBy`. Devolver los 10 primeros elementos del RDD resultante con `take` y almacenarlos en la lista `results`.

<div class="alert alert-block alert-danger">
<i class="fa fa-exclamation-triangle" aria-hidden="true"></i>
__OJO!__ Esta operación también requiere varios minutos.
</div>

In [19]:
results = similar_moviesRDD.sortBy(lambda x: x[0]).take(10)


results[:3]
#[((1, 78499), (0.9910368615281989, 37)),
# ((1, 3114), (0.9870973879980668, 101)),
# ((1, 80463), (0.9862946237042478, 25))]

[((1, 29), (0.975119822819981, 26)),
 ((1, 107), (0.9727726133019045, 22)),
 ((1, 150), (0.9717061436864811, 103))]

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 10 </font> 

Mostrar los resultados obtenidos.


In [22]:
print("Las 10 películas más parecidas a " + df_movies.loc[movie_id]['title']+"\n\n")
similar_movies_id = list(map(lambda x: x[0][1],results))
display(df_movies.loc[similar_movies_id]['title'])

Las 10 películas más parecidas a Toy Story (1995)




movieId
29     City of Lost Children, The (Cité des enfants p...
107                        Muppet Treasure Island (1996)
150                                     Apollo 13 (1995)
161                                  Crimson Tide (1995)
163                                     Desperado (1995)
247                            Heavenly Creatures (1994)
261                                  Little Women (1994)
348                         Bullets Over Broadway (1994)
364                                Lion King, The (1994)
376                               River Wild, The (1994)
Name: title, dtype: object

---

## 5. Librería MLib: ALS (Alternative Least Squares)

La librería _MLib_ contiene algoritmos de aprendizaje automáticos implementados para ejecutarse sobre la plataforma _Spark_. 

Como ejemplo, vamos entrenar un sistema de recomendación basado en _Alternative Least Squares_. Como se vio en el tema, este algoritmo optimiza tanto los parámetros que caracterizan las películas ($X$) como los que caracterizan a los usuarios $\theta$. Es decir, hace una factorización de matrices.

In [23]:
from pyspark.mllib.recommendation import ALS, Rating

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 11 </font> 

Transformar los datos del RDD `dataRDD` en otro RDD de objetos `Rating`. Cada uno de ellos se crea como `Rating(user_id, movie_id,rating)`.

In [25]:
ratingsRDD = dataRDD.map(lambda x: Rating(int(x.split(',')[0]),int(x.split(',')[1]),float(x.split(',')[2])))
ratingsRDD.take(3)

[Rating(user=1, product=31, rating=2.5),
 Rating(user=1, product=1029, rating=3.0),
 Rating(user=1, product=1061, rating=3.0)]

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 12 </font> 

Entrenar el modelo, con los parámetros proporcionados (mirar documentación). 

In [28]:
rank = 10
n_iterations = 6
model = ALS.trainImplicit(ratingsRDD,rank,iterations=n_iterations,seed=10)

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 13 </font> 

Obtener las valoraciones para el usuario 20 y almacenarlas en `user_ratingsRDD`.

In [34]:
user_id = 20
print("\nValoraciones para el usuario " + str(user_id) + ": \n")

# Filtra por usuario
user_ratingsRDD = ratingsRDD.filter(lambda x: x.user == user_id)


print("El usuario ha votado %d películas \n\n" % (user_ratingsRDD.count()))


Valoraciones para el usuario 20: 

El usuario ha votado 98 películas 




#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i>  Ejercicio 14 </font> 

Obtener e imprimir las 10 mejores recomendaciones para el usuario (a partir del objeto `model`) creado anteriormente.

In [104]:
print("\nMejores 10 recomendaciones \n ")
products = user_ratingsRDD.collect()
list_products = list(map(lambda x: x[1],products))
df_recomendations = pd.DataFrame(index=list_products)
df_recomendations['prediction'] = list(map(lambda x: model.predict(user_id, x), list_products))
df_recomendations.sort_values(by='prediction', ascending=False,inplace=True)
df_movies.loc[df_recomendations.index[:10]]


Mejores 10 recomendaciones 
 


,title,genres
356,Forrest Gump (1994),Comedy|Drama|Romance|War
318,"Shawshank Redemption, The (1994)",Crime|Drama
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
110,Braveheart (1995),Action|Drama|War
527,Schindler's List (1993),Drama|War


<div style="text-align: right"> <font size=5> [<i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#990003">](#principio)</i></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#990003"></i> </font></div>